## Start Milvus by Docker

In [7]:
%cd ./milvusDocker
!docker compose up -d
%cd ..

[Errno 2] No such file or directory: './milvusDocker'
/Users/xinkaiwang/Downloads/Cogno/Core-Service/databaseMod/milvusDocker
[+] Running 0/0
 ⠋ Network milvus  Creating                                                0.1s 
[+] Running 1/1
 ✔ Network milvus          Created                                         0.1s 
 ⠋ Container milvus-minio  Creating                                        0.0s 
 ⠋ Container milvus-etcd   Creating                                        0.0s 
[+] Running 1/3
 ✔ Network milvus          Created                                         0.1s 
 ⠙ Container milvus-minio  Creating                                        0.1s 
 ⠙ Container milvus-etcd   Creating                                        0.1s 
[+] Running 3/3
 ✔ Network milvus               Created                                    0.1s 
 ✔ Container milvus-minio       Created                                    0.2s 
 ✔ Container milvus-etcd        Created                                    0.2s 


In [8]:
import pandas as pd
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

from milvusDB import MilvusDB

### Use new database by setting param db_name instead of "default". It will be created automatically if it does not exist.

In [9]:
mvs_db = MilvusDB()
print(mvs_db.db_name)

mvs_db.db_name = 'product_db'
print(mvs_db.db_name)

default
product_db


In [10]:
# show current collections in used database
mvs_db.list_collections()

[]

### Read product parameters data, create a new collection named "product_stats" and insert data into collection

In [13]:
df = mvs_db.read_df_with_vector(
    csv_file_path="cognoSampleData/embeddedData/product_data.csv",
    embedded_col='product_intro_embed',
    index_col=None,
)

collection_name = "product_stats"

mvs_db.collection = mvs_db.create_collection_by_df(
    df=df,
    collection_name=collection_name,
    schema_description="product related parameters",
    pk_field_name='product_link',
)

mvs_db.insert_df_into_collection(df=df)

alloc_timestamp unimplemented, ignore it


Insert dataframe into collection product_stats successfully


In [14]:
mvs_db.collection.name

'product_stats'

### Search product info by query

In [15]:
res = mvs_db.conduct_vector_similar_search(query="新鲜红提", limit=5,
                                           output_fields=['product_name', 'product_intro', 'description'])
for search_res in res:
    for hit in search_res:
        print(hit)

id: https://item.jd.com/4225568.html, distance: 0.3221053183078766, entity: {'product_intro': '京鲜生 千禧圣女果 小西红柿 樱桃番茄 约1.5kg 水果礼盒 生鲜水果', 'description': '京鲜生 千禧圣女果 小西红柿 樱桃番茄 约1.5kg 水果礼盒 生鲜水果', 'product_name': '京鲜生 千禧圣女果 小西红柿 樱桃番茄 约1.5kg 水果礼盒 生鲜水果'}
id: https://item.jd.com/4746476.html, distance: 0.3494224548339844, entity: {'product_intro': '品牌： 京鲜生\n商品名称：京鲜生进口红玫瑰苹果\n商品编号：4746476\n商品毛重：2.1kg\n商品产地：新西兰\n品种：皇后\n单果果径：75mm-79mm\n贮存条件：控温\n国产/进口：进口', 'description': '主体\n总净含量\n1.5kg\n包装清单\n新西兰进口皇后红玫瑰苹果 特级果12粒礼盒装', 'product_name': ' 京鲜生新西兰皇后红玫瑰苹果 特级12粒礼盒装 单果重130-170g'}
id: https://item.jd.com/100064060629.html, distance: 0.35320645570755005, entity: {'product_intro': '品牌： 京鲜生\n商品名称：京鲜生克瑞森\n商品编号：100064060629\n商品毛重：2.36kg\n商品产地：云南\n品种：红提\n原产地：云南\n贮存条件：冷藏\n国产/进口：国产', 'description': '主体\n总净含量\n2kg\n包装清单\n无籽红提 精品2kg原框装', 'product_name': '京鲜生 国产克瑞森 无籽红提 精品2kg原框装 新鲜水果'}
id: https://item.jd.com/5383292.html, distance: 0.38638776540756226, entity: {'product_intro': '品牌： 京鲜生\n商品名称：京鲜生火龙果\n商品编号：5383292\n商品毛重

### create product review collection and insert data into it

In [16]:
product_review_sample_data = "cognoSampleData/embeddedData/product_review_JDFresh_product13.csv"

In [17]:
df = mvs_db.read_df_with_vector(product_review_sample_data, 'review_embed')
df.head()

,username,rev_score,review,product_link,review_embed
0,u***2,5,沉甸甸的很新鲜，每一个牛油果都是圆润又饱满颜值很高呀，皮薄肉厚而且很香。而且价格还那么便宜 ...,https://item.jd.com/8094996.html,"[-0.006060838645441247, -0.013074993039619596,..."
1,刘***6,5,买一送一简直不要太划算，吃轻食，减肥必备牛油果，放冰箱会熟得很慢，要吃前两天拿出来常温放置，...,https://item.jd.com/8094996.html,"[-0.007103661014159537, -0.01758937150018359, ..."
2,c***4,5,物流速度那是相当的快，而且买一送一，简直太划算了，家里人都很喜欢吃，而且家里小朋友从小就吃惯...,https://item.jd.com/8094996.html,"[-0.010681010629880971, -0.002514252868825385,..."
3,秋***7,5,品质很好，很喜欢这个，而且一盒里面有成熟的有半熟的，商家很贴心，慢慢吃也不担心会熟坏了，包装...,https://item.jd.com/8094996.html,"[-0.00576642902418351, -0.02692542419385611, -..."
4,j***5,5,京鲜生秘鲁进口牛油果价格超级便宜，性价比极高，值得推荐，味道尚可，脂肪含量极高，种植过程极为...,https://item.jd.com/8094996.html,"[0.006983776256257754, -0.0048000977965308965,..."


In [18]:
new_collection_name = "product_review"

mvs_db.collection = mvs_db.create_collection_by_df(
    df=df,
    collection_name=new_collection_name,
    schema_description="product review data",
    pk_field_name=None,
)

alloc_timestamp unimplemented, ignore it


In [19]:
mvs_db.insert_df_into_collection(df=df)

Insert dataframe into collection product_review successfully


### search similar review by query and get corresponding product parameters data from product_stats collection

In [23]:
mvs_db.collection = "product_review"
review_search_res = mvs_db.conduct_vector_similar_search(query="已购，孩子很喜欢", limit=5,
                                                         output_fields=['username', 'rev_score', 'review', 'product_link'])

mvs_db.collection = "product_stats"
for rev_res in review_search_res:
    for hit in rev_res:
        print("similar product review: ")
        print(hit)
        prod_link = hit.entity.get('product_link')
        prod_stat_res = mvs_db.query_search(expr=f'product_link in ["{prod_link}"]',
                                            output_fields=['product_name', 'product_intro', 'description'])
        if not prod_stat_res:
            print(f'no match product in collection {mvs_db.collection.name} of database {mvs_db.db_name}')
            continue
        print("product information: ")
        print(prod_stat_res[0])
        print('\n')

similar product review: 
id: 443608308898596255, distance: 0.2123182713985443, entity: {'review': '非常好，孩子挺喜欢吃的，也新鲜  ，快没有了，这是最后两个，物流也快 ，第二天就到了，下次有活动还买，孩子吃了挺好的，非常喜欢', 'product_link': 'https://item.jd.com/8094996.html', 'username': '小***哭', 'rev_score': 5}
product information: 
{'product_name': '京鲜生 秘鲁进口牛油果 宝宝果6粒装 单果110g起 生鲜 新鲜水果', 'product_intro': '品牌： 京鲜生\n商品名称：京鲜生进口牛油果\n商品编号：8094996\n商品毛重：0.7kg\n商品产地：秘鲁\n国产/进口：进口\n贮存条件：常温\n原产地：秘鲁', 'description': '主体\n总净含量\n0.66kg\n包装清单\n京鲜生 秘鲁进口牛油果 中小果6粒装 单果110g起 生鲜 新鲜水果', 'product_link': 'https://item.jd.com/8094996.html'}


similar product review: 
id: 443608308898596310, distance: 0.2550034523010254, entity: {'review': '早就计划买了趁着活动入手，，娃娃很喜欢，，味道不错，口感很好。。内容丰富，，送人也是可以的。很好很喜欢。用起来非常不错。', 'product_link': 'https://item.jd.com/8094996.html', 'username': 'n***i', 'rev_score': 5}
product information: 
{'description': '主体\n总净含量\n0.66kg\n包装清单\n京鲜生 秘鲁进口牛油果 中小果6粒装 单果110g起 生鲜 新鲜水果', 'product_link': 'https://item.jd.com/8094996.html', 'product_name': '京鲜生 秘鲁进口牛油果 宝

### Shut down Milvus

In [24]:
%cd ./milvusDocker
!docker compose down

/Users/xinkaiwang/Downloads/Cogno/Core-Service/databaseMod/milvusDocker
[+] Running 0/0
 ⠋ Container milvus-standalone  Stopping                                   0.1s 
[+] Running 0/1
 ⠙ Container milvus-standalone  Stopping                                   0.2s 
[+] Running 0/1
 ⠹ Container milvus-standalone  Stopping                                   0.3s 
[+] Running 0/1
 ⠸ Container milvus-standalone  Stopping                                   0.4s 
[+] Running 0/1
 ⠼ Container milvus-standalone  Stopping                                   0.5s 
[+] Running 0/1
 ⠴ Container milvus-standalone  Stopping                                   0.6s 
[+] Running 0/1
 ⠦ Container milvus-standalone  Stopping                                   0.7s 
[+] Running 0/1
 ⠧ Container milvus-standalone  Stopping                                   0.8s 
[+] Running 0/1
 ⠇ Container milvus-standalone  Stopping                                   0.9s 
[+] Running 0/1
 ⠏ Container milvus-standalone  Stoppin

In [25]:
# remove used database if necessary
!rm -rf volumes/